In [1]:
from multiprocess import Pool
import itertools
import numpy as np

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

In [2]:
from datasets import load_dataset
ds = load_dataset("diarizers-community/synthetic-speaker-diarization-dataset")

print(ds)

README.md:   0%|          | 0.00/459 [00:00<?, ?B/s]

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


train-00000-of-00006.parquet:   0%|          | 0.00/442M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/427M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/431M [00:00<?, ?B/s]

train-00003-of-00006.parquet:   0%|          | 0.00/444M [00:00<?, ?B/s]

train-00004-of-00006.parquet:   0%|          | 0.00/436M [00:00<?, ?B/s]

train-00005-of-00006.parquet:   0%|          | 0.00/431M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1584 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'speakers', 'timestamps_start', 'timestamps_end'],
        num_rows: 1584
    })
})


In [3]:
import string
import soundfile as sf
import numpy as np
from collections import defaultdict

def convert_rttm(chunk, filename = 'audio'):
    rttm = []
    for start, end, speaker in chunk:
        duration = end - start
        rttm.append(f"SPEAKER {filename} 1 {start:.4f} {duration:.4f} <NA> <NA> <NA> <NA> {speaker}")
    return '\n'.join(rttm)

def convert_textgrid(segments):
    tiers = defaultdict(list)
    for start, end, speaker in segments:
        tiers[speaker].append((start, end))

    min_time = min(start for start, _, _ in segments)
    max_time = max(end for _, end, _ in segments)

    textgrid = []
    textgrid.append("File type = \"ooTextFile\"")
    textgrid.append("Object class = \"TextGrid\"")
    textgrid.append("")
    textgrid.append(f"xmin = {min_time:.2f}")
    textgrid.append(f"xmax = {max_time:.2f}")
    textgrid.append("tiers? <exists>")
    textgrid.append(f"size = {len(tiers)}")
    textgrid.append("item []:")

    for i, (speaker, intervals) in enumerate(tiers.items(), start=1):
        textgrid.append(f"    item [{i}]:")
        textgrid.append("        class = \"IntervalTier\"")
        textgrid.append(f"        name = \"{speaker}\"")
        textgrid.append(f"        xmin = {min_time:.2f}")
        textgrid.append(f"        xmax = {max_time:.2f}")
        textgrid.append(f"        intervals: size = {len(intervals)}")

        for j, (start, end) in enumerate(intervals, start=1):
            textgrid.append(f"        intervals [{j}]:")
            textgrid.append(f"            xmin = {start:.2f}")
            textgrid.append(f"            xmax = {end:.2f}")
            textgrid.append(f"            text = \"{speaker}\"")
            
    return '\n'.join(textgrid)

timestamps = [i * 0.02 for i in range(1500 + 1)]

In [4]:
# !rm -rf synthetic-speaker-diarization-dataset
!mkdir synthetic-speaker-diarization-dataset

In [5]:
from tqdm import tqdm
import os

def loop(indices):
    indices, _ = indices
    ds = load_dataset("diarizers-community/synthetic-speaker-diarization-dataset")
    data = []
    for k, key in tqdm(indices):
        row = ds[key][k]
        audio = row['audio']['array']
        chunks, temp = [], []
        argsort = np.argsort(row['timestamps_start'])
        timestamps_start = [row['timestamps_start'][i] for i in argsort]
        timestamps_end = [row['timestamps_end'][i] for i in argsort]
        speakers = [row['speakers'][i] for i in argsort]
        start = timestamps_start[0]
        max_len = 30
        for i in range(len(timestamps_start)):
            l = timestamps_end[i] - start
            if l >= max_len:
                chunks.append(temp)
                temp = [[timestamps_start[i], timestamps_end[i], speakers[i]]]
                start = timestamps_start[i]
                continue
            else:
                temp.append([timestamps_start[i], timestamps_end[i], speakers[i]])

        if len(temp):
            chunks.append(temp)

        for no, chunk in enumerate(chunks):
            speakers = []
            for i in range(len(chunk)):
                if chunk[i][-1] not in speakers:
                    speakers.append(chunk[i][-1])
            
            try:          
                start_time = chunk[0][0]
                end_time = max([c[1] for c in chunk])
            except Exception as e:
                continue
                
            if round(end_time - start_time, 2) > max_len:
                continue
            
            y = audio[int(16000 * start_time): int(16000 * end_time)]
            audio_filename = f'synthetic-speaker-diarization-dataset/{key}-{k}-{no}.mp3'
            if not os.path.exists(audio_filename):
                sf.write(audio_filename, y, 16000)
            
            ts = []
            for i in range(len(chunk)):
                index = speakers.index(chunk[i][-1])
                start = min(timestamps, key=lambda t: abs(t - (chunk[i][0] - start_time)))
                end = min(timestamps, key=lambda t: abs(t - (chunk[i][1] - start_time)))
                speaker_name = f'speaker {string.ascii_uppercase[index]}'
                chunk[i][-1] = speaker_name
                chunk[i][0] = start
                chunk[i][1] = end
                t = f"<|{start:.2f}|> {speaker_name}<|{end:.2f}|>"
                ts.append(t)
                
            ts = ''.join(ts)
            rttm = convert_rttm(chunk)
            textgrid = convert_textgrid(chunk)
            
            data.append({
                'question': 'diarize the audio using whisper format',
                'answer': ts,
                'audio_filename': audio_filename,
            })
            data.append({
                'question': 'diarize the audio using rttm format',
                'answer': rttm,
                'audio_filename': audio_filename,
            })
            data.append({
                'question': 'diarize the audio using textgrid format',
                'answer': textgrid,
                'audio_filename': audio_filename,
            })
            
    return data

In [10]:
indices = list(range(len(ds['train'])))
indices = [(i, 'train') for i in indices]
prepared = multiprocessing(indices, loop, cores = min(len(indices), 20))

100%|███████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:26<00:00,  2.95it/s]


In [13]:
len(prepared)

10878

In [14]:
import pandas as pd

pd.DataFrame(prepared).to_parquet('synthetic-speaker-diarization-dataset.parquet')

In [18]:
!huggingface-cli upload mesolitica/Speaker-Diarization-Instructions \
synthetic-speaker-diarization-dataset.parquet /data/synthetic_speaker_diarization_dataset-00000-of-00001.parquet \
--repo-type=dataset

Uploading files using Xet Storage..
Uploading...: 100%|█████████████████████████| 1.89M/1.89M [00:04<00:00, 403kB/s]
https://huggingface.co/datasets/mesolitica/Speaker-Diarization-Instructions/blob/main//data/synthetic_speaker_diarization_dataset-00000-of-00001.parquet
